In [3]:
import os
import torch
import torchvision
from torchvision import transforms , datasets

from torch.utils.data import DataLoader , random_split
import torch.nn as nn 
import torch.optim as optim
import numpy as np 
import pandas

In [4]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
    
])

In [5]:
train_data=datasets.ImageFolder('./dataset/Training',transform=transform)
test_data=datasets.ImageFolder('./dataset/Testing',transform=transform)

In [6]:
train_data

Dataset ImageFolder
    Number of datapoints: 5712
    Root location: ./dataset/Training
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )

In [7]:
train_size=int(0.8 * len(train_data))
val_size=len(train_data)-train_size

train_dataset,val_dataset=random_split(train_data,[train_size,val_size])

In [8]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=16,shuffle=True)
test_loader=DataLoader(test_data,batch_size=16,shuffle=True)


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),    # First convolutional layer with 32 filters (kernels)
            nn.ReLU(),                         # ReLU activation function to amplify important signal
            nn.MaxPool2d(2),                   # Decreasing dimension of feature maps by the factor of 2
            nn.Conv2d(32, 64, 3, padding=1),   # Another convolutional layer with 64 filters
            nn.ReLU(),                         # ReLU activation function to amplify important signal
            nn.MaxPool2d(2)                    # Decreasing dimensions of feature map agaub by the f actor of 2
        )

        self.fc1 = nn.Sequential(
            # Mapping this tensor:
            # [
            #      <image>: [1 x 200704 vector],
            #      <image>: [1 x 200704 vector],
            #               .
            #               .
            #               .
            # ]
            #  to this tensor with preservation of features:
            # [
            #      <image>: [1 x 128 vector],
            #      <image>: [1 x 128 vector],
            #               .
            #               .
            #               .
            # ]
            
            nn.Linear(64 * 56 * 56, 128),
            nn.ReLU(),
            nn.Dropout(0.15),                      # Randomization of feature maps to prevent overfitting
            nn.Linear(128, 4)                      # Getting logits for each image with 1 x 4 vector, each element in this vector represents
                                                   # raw probability
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)          # getting shape ( batch_size, 64 * 56 * 56 ) 
        x = self.fc1(x)
        return torch.softmax(x, dim=1)     # outputting probabilities that sum up to 1


In [12]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=CNN().to(device)

for name, param in model.named_parameters():
    print(name, param.shape, param.requires_grad, param.grad)


conv.0.weight torch.Size([32, 3, 3, 3]) True None
conv.0.bias torch.Size([32]) True None
conv.3.weight torch.Size([64, 32, 3, 3]) True None
conv.3.bias torch.Size([64]) True None
fc1.0.weight torch.Size([128, 200704]) True None
fc1.0.bias torch.Size([128]) True None
fc1.3.weight torch.Size([4, 128]) True None
fc1.3.bias torch.Size([4]) True None


In [16]:
crit=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [18]:
for epoch in range(10):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = crit(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Optional: Track accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/10] - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch [1/10] - Loss: 1.4703, Accuracy: 27.34%
Epoch [2/10] - Loss: 1.4702, Accuracy: 27.34%
Epoch [3/10] - Loss: 1.4700, Accuracy: 27.34%
Epoch [4/10] - Loss: 1.4702, Accuracy: 27.34%
Epoch [5/10] - Loss: 1.4702, Accuracy: 27.34%
Epoch [6/10] - Loss: 1.4704, Accuracy: 27.34%
Epoch [7/10] - Loss: 1.4703, Accuracy: 27.34%
Epoch [8/10] - Loss: 1.4706, Accuracy: 27.34%
Epoch [9/10] - Loss: 1.4704, Accuracy: 27.34%
Epoch [10/10] - Loss: 1.4702, Accuracy: 27.34%


In [13]:
from torchvision.models import resnet50

In [14]:
model=resnet50().to(device)
model.fc=nn.Linear(model.fc.in_features,4).to(device)

crit=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [15]:
for epoch in range(10):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = crit(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Optional: Track accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/10] - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch [1/10] - Loss: 1.6174, Accuracy: 48.06%
Epoch [2/10] - Loss: 0.8384, Accuracy: 67.08%
Epoch [3/10] - Loss: 0.7132, Accuracy: 71.59%
Epoch [4/10] - Loss: 0.7121, Accuracy: 73.25%
Epoch [5/10] - Loss: 0.6152, Accuracy: 77.11%
Epoch [6/10] - Loss: 0.5542, Accuracy: 78.77%
Epoch [7/10] - Loss: 0.5119, Accuracy: 80.70%
Epoch [8/10] - Loss: 0.4972, Accuracy: 81.27%
Epoch [9/10] - Loss: 0.4519, Accuracy: 83.32%
Epoch [10/10] - Loss: 0.4230, Accuracy: 84.15%


In [17]:
# Dump the model
torch.save(model.state_dict(), 'resnet50_trained.pth')